ETF Information

In [46]:
import pandas as pd
from pathlib import Path
import numpy as np


#importing matplotlib setting style for plotting later
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg' 
plt.style.use('fivethirtyeight')

# Reading datasets
database_path= r'database'
# Get the files from the path
files = Path(database_path).glob('*.csv')
#Add new column with file name
dfs = list()
for f in files:
    data = pd.read_csv(f)
    data['ETF_name'] = f.stem
    dfs.append(data)
    
etf = pd.concat(dfs, ignore_index=True)


#Selecting and printing columns
print(etf.columns)

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends',
       'Stock Splits', 'ETF_name'],
      dtype='object')


Presenting unique ETF

In [47]:
#Selecting unique ETF and setting the index
unique_etf = etf['ETF_name'].unique()
print(unique_etf)
print(etf.head())


['ARKF' 'BITQ' 'BLOK' 'EMFQ' 'FINX' 'FNTC' 'IPAY' 'IYG' 'KOIN' 'TPAY']
                        Date       Open       High        Low      Close  \
0  2019-02-04 00:00:00-05:00  19.837522  20.073683  19.834570  20.073683   
1  2019-02-05 00:00:00-05:00  20.152404  20.280325  20.103204  20.204557   
2  2019-02-06 00:00:00-05:00  20.191766  20.205542  19.985124  20.142565   
3  2019-02-07 00:00:00-05:00  19.965444  19.994963  19.680082  19.847363   
4  2019-02-08 00:00:00-05:00  19.680081  19.886721  19.630882  19.867043   

   Volume  Dividends  Stock Splits ETF_name  
0   47200        0.0             0     ARKF  
1  110700        0.0             0     ARKF  
2   45300        0.0             0     ARKF  
3   58200        0.0             0     ARKF  
4   18900        0.0             0     ARKF  


In [50]:
# plot
COLORS = ('orange','green', 'blue', 'red', 'yellow', 'brown', 'darkgreen', 'darkblue')
etf_2 = etf.set_index('ETF_name')
Volume_sum = etf["Volume"].sum()
ax = etf_2.plot.bar(y=Volume_sum, color=COLORS, legend=False, grid=True)
ax.set_ylabel('Total Volume')
ax.set_title('ETF Volume by ETF Name')
plt.xticks(rotation=45)
plt.show()


KeyError: "None of ['ETF_name'] are in the columns"